# Calcul du $F_{ST}$ à partir d'un jeu de données en format .ped

Nous allons calculer la valeur du $F_{ST}$ à partir d'un dataset sous format ped (voir la description du format ped [ici](http://www.shapeit.fr/pages/m02_formats/pedmap.html)). 

Les fichiers de données peut être téléchargé [ici](https://housecloud.willyrv.com/index.php/s/kN56oKNgkbmPxUf)

In [1]:
#### Quelques fonctions utiles  ####

getGenotypesLine <- function(pedSnpsLine){
  return(unlist(lapply(seq(1, length(pedSnpsLine), 2), 
                       FUN = function(i) return(paste(pedSnpsLine[i], pedSnpsLine[i+1], sep = "")))))
}

getGenotypes <- function(pedTable){
  return(t(apply(pedTable[, -c(1:6)], MARGIN = 1, getGenotypesLine)))
}

freqIBS <- function(genotypesData){
  tempIBS <- 0
  nbMrks <- dim(genotypesData)[2]-1
  for (i in 2:nbMrks) {  # boucle sur les colonnes
    mrkColumn <- genotypesData[, i]
    alleles <- paste(mrkColumn, collapse = "")
    allelesVect <- strsplit(alleles, split = "")[[1]]
    allelesCount <- table(allelesVect)
    allelesCountNoMissing <- allelesCount[!grepl("0", names(allelesCount))]
    if (length(allelesCountNoMissing > 0)) { # ne pas prendre en compte les markeurs ayant que de zéros
      p <- allelesCountNoMissing[1]/sum(allelesCountNoMissing)
      mrkIBS <- 1 - 2 * p * (1 - p)
      tempIBS <- tempIBS + mrkIBS
    }
  }
  IBS <- as.numeric(tempIBS) / nbMrks
  return(IBS)
}

computeFst <- function(genotypesData){
  ####  Calculer la frequence d'IBS dans la population total
  fBar <- freqIBS(genotypesData)
  
  ####  Calculer la frequence d'IBS à l'intérieur d'une sous-population
  populations <- unique(crossData[, 1])
  f0Temp <- 0
  for (p in populations) {
    subPopulation <- myData[myData[, 1] == p, ]
    f0Temp <- f0Temp + freqIBS(subPopulation)
  }
  f0 <- f0Temp / length(populations)
  
  #### Calcul du Fst 
  Fst <- (f0 - fBar) / (1 - fBar)
  return(Fst)
}


In [2]:
# Lire les données en format ped 
crossData <- read.table("./dataSet_1.ped", stringsAsFactors = FALSE)

myData <- cbind.data.frame(crossData[, 1], getGenotypes(crossData))

In [3]:
# Calcul du F_{ST}
computeFst(myData)

[1] 0.1569435

In [4]:
# Lire les données en format ped 
crossData <- read.table("./dataSet_2.ped", stringsAsFactors = FALSE)

myData <- cbind.data.frame(crossData[, 1], getGenotypes(crossData))


# Calcul du F_{ST}
computeFst(myData)

[1] 0.1505492

In [5]:
# Lire les données en format ped 
crossData <- read.table("./NEU_short.ped", stringsAsFactors = FALSE)

myData <- cbind.data.frame(crossData[, 1], getGenotypes(crossData))


# Calcul du F_{ST}
computeFst(myData)

[1] 0.1148371

La valeur du $F_{ST}$ indique qu'il y a un certain niveau de structuration dans la population